In [ ]:
import os
import pandas as pd
from PIL import Image
import torch

from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


from torch.utils.data import DataLoader
from torchvision import transforms

#### create path to my data on pc

In [ ]:
train_path = r"C:\Users\1\Desktop\University_semeter_3\ZNEUS\archive\ASL_Dataset\Train"

#### create csv file with image pases and labels

In [ ]:
data = []

for folder in os.listdir(train_path):
    folder_path = os.path.join(train_path, folder)

    if os.path.isdir(folder_path):
        for image in os.listdir(folder_path):
            if image.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(folder_path, image)
                data.append({'path': image_path, 'label': folder})

# Створюємо DataFrame
df = pd.DataFrame(data)

# Зберігаємо в CSV
df.to_csv('dataset.csv', index=False)
print(f" {len(df)} rows")
print(f"Clases: {df['label'].unique()}")


In [ ]:
ds = pd.read_csv('dataset.csv')
ds.head()


In [ ]:
ds.info()

In [ ]:
ds.duplicated().sum()

In [ ]:
ds = ds.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
ds_answer =ds.iloc[:,-1:]
ds_answer.head()

In [ ]:
ds_train = ds.iloc[:,:-1]
ds_train.head()

In [ ]:
for i in range(3):
    img = Image.open(ds_train.iloc[i,0])
    img.show()




In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor()
])

img_path = df.iloc[0]['path']
img = Image.open(img_path)

img_tensor = transform(img)

print(img_tensor.shape)

In [ ]:
# make dataset with tensors
class ASLDataset(Dataset):
    def __init__(self, csv_file, transform):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

        unique_labels = sorted(self.data['label'].unique())
        self.label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = Image.open(self.data.iloc[idx]['path'])
        label = self.data.iloc[idx]['label']
        label_idx = self.label_to_idx[label]
        return self.transform(img), label_idx


dataset = ASLDataset('dataset.csv', transform=transform)
loader = DataLoader(
    dataset,
    batch_size=512,
    shuffle=True)

In [159]:
conv1 = nn.Conv2d(1,6,3,1)
conv2 = nn.Conv2d(6,12,3,1)
conv3 = nn.Conv2d(12,18,3,1)
conv4 = nn.Conv2d(18,24,3,1)
conv5 = nn.Conv2d(24,32,3,1)

In [162]:
x = F.relu(conv1(img_tensor.unsqueeze(0)))
x.shape

torch.Size([1, 6, 398, 398])

In [163]:
x = F.max_pool2d(x,2,2)
x.shape

torch.Size([1, 6, 199, 199])

In [164]:
x = F.relu(conv2(x))
x.shape

torch.Size([1, 12, 197, 197])

In [165]:
x = F.max_pool2d(x,2,2)
x.shape

torch.Size([1, 12, 98, 98])

In [166]:
x = F.relu(conv3(x))
x.shape

torch.Size([1, 18, 96, 96])

In [167]:
x = F.max_pool2d(x,2,2)
x.shape

torch.Size([1, 18, 48, 48])

In [168]:
x = F.relu(conv4(x))
x.shape

torch.Size([1, 24, 46, 46])

In [169]:
x = F.max_pool2d(x,2,2)
x.shape

torch.Size([1, 24, 23, 23])

In [170]:
x = F.relu(conv5(x))
x.shape

torch.Size([1, 32, 21, 21])

In [171]:
x = F.max_pool2d(x,2,2)
x.shape

torch.Size([1, 32, 10, 10])

In [172]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,3,1)
        self.conv2 = nn.Conv2d(6,12,3,1)
        self.conv3 = nn.Conv2d(12,18,3,1)
        self.conv4 = nn.Conv2d(18,24,3,1)
        self.conv5 = nn.Conv2d(24,32,3,1)
        self.fc1 = nn.Linear(32*10*10,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,29)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x,2,2)
        x= F.relu(self.conv4(x))
        x = F.max_pool2d(x,2,2)
        x= F.relu(self.conv5(x))
        x= F.max_pool2d(x,2,2)
        x = x.view(-1,32*10*10)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x= self.fc3(x)
        return F.log_softmax(x, dim=1)

In [173]:
model = ConvolutionalNetwork()
model



ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(12, 18, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(18, 24, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=3200, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=29, bias=True)
)

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
Number_of_epochs = 2
for epoch in range(Number_of_epochs):
    for i, (images, labels) in enumerate(loader):
        # Перевір shape (тільки раз)
        if i == 0 and epoch == 0:
            print(f"Images: {images.shape}, Labels: {labels.shape}")

        y_pred = model(images)
        loss = loss_func(y_pred, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f'Epoch [{epoch+1}/10], Step [{i+1}], Loss: {loss.item():.4f}')

Images: torch.Size([32, 1, 400, 400]), Labels: torch.Size([32])
Epoch [1/10], Step [1], Loss: 3.3541
Epoch [1/10], Step [101], Loss: 3.3371
Epoch [1/10], Step [201], Loss: 2.9925
Epoch [1/10], Step [301], Loss: 1.6433
Epoch [1/10], Step [401], Loss: 0.7734
Epoch [1/10], Step [501], Loss: 0.4346
Epoch [1/10], Step [601], Loss: 0.1265
Epoch [1/10], Step [701], Loss: 0.4347
Epoch [1/10], Step [801], Loss: 0.3198
Epoch [1/10], Step [901], Loss: 0.4645
